#### D'NOVA YouTube Shorts Comments & Transcript Crawling

##### Libraries

In [1]:
import time
import random
import pandas as pd
import pickle
import undetected_chromedriver as uc
import random
import re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from datetime import datetime, timedelta


##### Parameters

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
# Define the path where the Excel files are stored
db_path = 'DB'  # Replace with the correct path to the folder containing the files

# Get a list of all Excel files in the directory
files = [f for f in os.listdir(db_path) if f.startswith('loaded_db_100_') and f.endswith('.xlsx')]
# Extract the dates from the filenames
dates = [f.split('_')[3].split('.')[0] for f in files]

# Find the most recent date
latest_date = max(dates)

# Build the full file path for the most recent file
latest_file = f'loaded_db_100_{latest_date}.xlsx'
latest_file_path = os.path.join(db_path, latest_file)

##### Functions

In [5]:

# 우회를 위해 random하게 시간 배정
def random_sleep():
    """1~2초 사이의 랜덤한 대기"""
    time.sleep(random.uniform(1, 3))

# comments 끝까지 수집을 위해 scroll down
def scroll_to_bottom(driver):
    """페이지를 끝까지 스크롤 다운하는 함수"""
    
    # Perform a single scroll down to load comments
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        random_sleep()
        
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_youtube_comments(url):
    # uc driver로 브라우저 실행
    driver = uc.Chrome()
    driver.get(url)
    random_sleep()  # 페이지가 로드될 시간을 줍니다.
    
    # 페이지 끝까지 스크롤 다운
    scroll_to_bottom(driver)
    
    # 모든 댓글 수집
    comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
    comments = [comment.text for comment in comments_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 댓글 작성 날짜 수집
    date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
    dates = [date.text for date in date_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 결과를 리스트의 튜플 형태로 반환
    comments_data = list(zip(comments, dates))
    
    # 드라이버 종료
    driver.quit()
    
    return comments_data

# Define the youtube_transcript function
def youtube_transcript_music(driver):
    """Extract transcript text from YouTube Shorts video page."""
    music_texts = ''
    try:
        driver.implicitly_wait(15)  # Wait for page to load
        # Step 1: Click the 'more' button
        more_button = driver.find_element(By.XPATH, '//*[@id="expand"]')
        more_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay to ensure smooth loading
        driver.implicitly_wait(10)  # Wait for page to load
        # music 확인
        try:
            # Find all elements matching the XPath
            music_elements = driver.find_elements(By.XPATH, '//*[@id="items"]/yt-video-attribute-view-model/div/a')
            # Extract text from each element
            music_texts = [element.text for element in music_elements][0]
        except:
            print("No music")
            music_texts = ''
        # Step 2: Click the 'show transcript' button
        transcript_button = driver.find_element(By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        transcript_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay for loading transcript

        # Step 3: Collect all transcript text
        transcript_elements = driver.find_elements(By.XPATH, '//*[@id="body"]/ytd-transcript-segment-list-renderer')
        transcript_texts = [element.text for element in transcript_elements]
        
        # Combine all transcript texts into a single string
        transcript_data = " ".join(transcript_texts) if transcript_texts else "No transcript found"
        transcript_data = process_transcript(transcript_data)
        return transcript_data, music_texts  # Return the combined transcript text

    except Exception as e:
        print(f"Error occurred while extracting transcript or music: {e}")
        return '', music_texts
    

def process_transcript(transcript_text):
    # Remove time markers (e.g., "0:00")
    transcript_text = re.sub(r'\d+:\d+\s*', '', transcript_text)
    
    # Split by newline or sentence-ending punctuation
    sentences = re.split(r'(?<=[.?!])\s+', transcript_text.strip())
    
    # Remove any empty entries and return as a list of sentences
    return [sentence for sentence in sentences if sentence]

# 완료되는 부분까지 데이터 저장
def save_progress(df, output_file):
    """현재 진행 상태를 엑셀 파일로 저장하는 함수"""
    try:
        # Load existing data, if any
        existing_df = pd.read_excel(output_file) if os.path.exists(output_file) else pd.DataFrame()
        
        # Append new data
        df = pd.concat([existing_df, df], ignore_index=True)
        
        # Remove duplicate rows if needed
        df.drop_duplicates(subset=['channel_name', 'shorts_url', 'check_date'], keep='last', inplace=True)
        
        # Save the updated DataFrame back to the file
        df.to_excel(output_file, index=False)
        print(f"Progress saved to {output_file}")
    except Exception as e:
        print(f"Error saving progress: {e}")

# URL 접속 후 comments, transcript, music 수집 진행
def process_youtube_comments_transcript_music(driver, top_100):
    """Process YouTube video comments and save progress whenever an error occurs or the program ends."""
    
    for index, row in top_100.iterrows():
        random_sleep()        
        try:
            url = row['shorts_url']
            title = row['channel_name']
            driver.get(url)
            driver.implicitly_wait(10)
            
            # 수집할 script 및 comments
            transcript_music_data = youtube_transcript_music(driver)  # your youtube_transcript function should be defined
            random_sleep()
            # 여기서 comment num 확인 
            scroll_to_bottom(driver)
            
            comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
            comments = [comment.text for comment in comments_elements]
            random_sleep()
            
            date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
            dates = [date.text for date in date_elements]
            random_sleep()
            
            comments_data = list(zip(comments, dates)) if comments else [(None, None)]
            
            # Append data to DataFrame
            top_100.at[index, 'shorts_comments_time'] = [comments_data]
            top_100.at[index, 'transcript_data'] = transcript_music_data[0]
            top_100.at[index, 'music'] = transcript_music_data[1]


        except Exception as err:
            print(f"Error processing {title or 'Unknown'} at row {index}: {err}")
            save_progress(top_100, latest_file_path)
            raise
    
    save_progress(top_100, "loaded_db_100_"+today+"xlsx")
    return top_100


# Main function to run the transcript extraction example
def main(df):
    # YouTube Shorts URL to scrape
    options = uc.ChromeOptions()
    options.headless = False
    options.add_experimental_option('prefs', {'intl.accept_languages': 'en'})
    
    driver = uc.Chrome(options=options, use_subprocess=True)

    # Call the youtube_transcript function to get the transcript text
    # transcript_text = youtube_transcript(driver)
    data = process_youtube_comments_transcript_music(driver, df)
    # Close the driver
    driver.quit()
    return data

#### Load Data

In [9]:
# Load the most recent file into a DataFrame
top_100_df = pd.read_excel(latest_file_path)


In [9]:
top_100_df

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date,transcript_data,music
0,Marc Brunet,https://www.youtube.com/watch?v=Qc6NO19u3MM,NaN,👨‍🎨 Serious about your art? Check out my ART S...,https://support.google.com/youtube/answer/3037...,"121,372",17000,94,[[('The audacity of assuming I can draw those ...,NaN,2024-10-24 21:44:30.453,"[""what most artists struggle with when\ndrawin...",NaN
1,Jen_ny69,https://www.youtube.com/watch?v=ixq_pOK7jk0,NaN,Watch full video on ‪@adayinthelifeofa69‬,https://support.google.com/youtube/answer/3037...,"6,270",328,13,[[('Sweet boy. he’s well-loved and it shows.'...,NaN,2024-10-24 21:44:30.453,"[""what's up YouTube Welcome Back to the\nchann...",NaN
2,DawateIslami,https://www.youtube.com/watch?v=JhocFEigq3k,NaN,Kabhi Ye Bhi Try Karen | #dawateislami #whatsa...,https://support.google.com/youtube/answer/3037...,926,229,3,"[[('', '2 weeks ago'), ('Very very very very v...",NaN,2024-10-24 21:44:30.453,['बैर मुल्क छुटियां गुजारना एक ऑसम\nएक्सपीरियं...,NaN
3,Go Natural English with Gabby Wallace,https://www.youtube.com/watch?v=v9dcf7OF1Wg,NaN,NaN,https://support.google.com/youtube/answer/3037...,"3,897",72,3,"[[(' nice, in Colombia many many nice restaura...",NaN,2024-10-24 21:44:30.453,"[""I lived abroad outside the US for many\nyear...",NaN
4,ReligionForBreakfast,https://www.youtube.com/watch?v=WZn6dF9C1NQ,NaN,NaN,https://support.google.com/youtube/answer/3037...,"82,627",4700,206,"[[(""I'm pretty sure the Catholic Church has a ...",NaN,2024-10-24 21:44:30.453,"[""this Mustachio fellow is Jesus mde a\npopula...",NaN
5,Medicosis Perfectionalis,https://www.youtube.com/watch?v=wFVlzZUWFcw,NaN,NaN,https://support.google.com/youtube/answer/3037...,"2,977",284,5,[[('السلام عليكم ورحمة الله وبركاته \nجزاك الل...,NaN,2024-10-24 21:44:30.453,"[""Intro\nCardiology in 1 minute with meosis\np...",NaN
6,理科太太 Li Ke Tai Tai,https://www.youtube.com/watch?v=E1789fuBkkU,貓咪個性這麼差，原來都是基因決定的！,NaN,https://rr3---sn-vgqsrnsr.googlevideo.com/vide...,"15,447",327,12,"[[('兔子非常嗜甜，個性也特好', '9 months ago'), ('這個循環很是巧妙...",2024-01-17 22,2024-10-24 21:44:30.453,NaN,NaN
7,Brian Tracy,https://www.youtube.com/watch?v=C6HaBSkA6xU,Shift Your Mindset,Imagine the possibilities if failure weren’t a...,https://i.ytimg.com/vi/C6HaBSkA6xU/hq2.jpg?sqp...,"4,477",562,8,"[[('It is purpose that created us, that binds ...",2024-10-23 09,2024-10-24 21:44:30.453,"[""what one great thing would you dare to\ndrea...","stellar (Slowed + Reverb)\n.diedlonely, énouem..."
8,Now You See It,https://www.youtube.com/watch?v=g6lcT4EhNzM,The Best Steven Spielberg Interview Question E...,Movies: The Fablemans (2022) and Close Encount...,https://rr1---sn-f58xn2xxq-aj5l.googlevideo.co...,"47,959",2400,32,[[('You know you’re a good interviewer when yo...,2023-01-07 09,2024-10-24 21:44:30.453,"[""your father was a computer scientist\nyour m...",NaN


##### Excute

In [5]:
# Run the main function
data = main(top_100_df)

No music
No music
No music
No music
No music
No music
No music
Error occurred while extracting transcript or music: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]"}
  (Session info: chrome=131.0.6778.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D433E3+25059]
	(No symbol) [0x00CCCDE4]
	(No symbol) [0x00BABEC3]
	(No symbol) [0x00BEFD86]
	(No symbol) [0x00BEFFCB]
	(No symbol) [0x00C2D952]
	(No symbol) [0x00C11F44]
	(No symbol) [0x00C2B51E]
	(No symbol) [0x00C11C96]
	(No symbol) [0x00BE3FAC]
	(No symbol) [0x00BE4F3D]
	GetHandleVerifier [0x01035543+3113795]
	GetHandleVerifier [0x0104A20A+3198986]
	GetHandleVerifier [0x010429E2+3168226]
	GetHandleVerifier [0x00DE3250+680016]
	(No symbol) [0x00CD572D]
	(No symbol) [0x

##### Save Data

In [ ]:
data.to_excel("DB/loaded_db_100_" + today + ".xlsx", index=False)

### Prototype Test

In [22]:
options = uc.ChromeOptions()
options.headless = False
options.add_experimental_option('prefs', {'intl.accept_languages': 'en'})    
driver = uc.Chrome(options=options, use_subprocess=True)

#url = "https://www.youtube.com/watch?v=MmX6n65Lyxw"
url = "https://www.youtube.com/watch?v=jk82SEkpDM0"
title = "유트루 추천 쿠션! 헤라 리플렉션 스킨 글로우 쿠션"
driver.get(url)
driver.implicitly_wait(10)

# 수집할 script 및 comments
transcript_music_data = youtube_transcript_music(driver)  # your youtube_transcript function should be defined
random_sleep()
# 여기서 comment num 확인 
scroll_to_bottom(driver)

comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
comments = [comment.text for comment in comments_elements]
random_sleep()

date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
dates = [date.text for date in date_elements]
random_sleep()
driver.close()
comments_data = list(zip(comments, dates)) if comments else [(None, None)]

No music


In [24]:
print("제목: ", title)
print("comments: ", comments_data)
print("Transcript: ", transcript_music_data[0])

제목:  유트루 추천 쿠션! 헤라 리플렉션 스킨 글로우 쿠션
comments:  [('다른 유튜버 리뷰 다 나와도 안 보고 언니꺼만 기다렸는데 촉촉쿠션 처돌이는 기뻐서 환장합니다.', '6 days ago'), ('헤라 글로우 쿠션 리뷰 유트루님꺼만 기다렸어요...\n광고여도 유트루님이 선택한 제품이라는거니 더 믿음가버리는..', '6 days ago'), ('6:48 이거 구매포인트다 트루언니가 코는 브러쉬 사용하는거 맨날 봤었는데 그러지 않아도 되는 쿠션이다?? 당장 사야지', '5 days ago'), ('쿠션류들은 N시간후 수정화장까지 보여주시면 좋을것같아요', '6 days ago'), ('유트루 언니에게 홀려 에스쁘아도 헤라도 둘다 구매해버려따..', '6 days ago'), ('이게 무신일이여 영상의 축복의 끝이 없네', '6 days ago'), ('지금쓰고잇는데 정말조아요 에스쁘아보다 헤라 글로우 잘맞는거같아용 인생템 찾앗어여', '6 days ago (edited)'), ('21호도 밝아보여요..23호쓰시면 훨씬 고급스러울거같아요 나중에 23호 사용도 보여주세요!!', '3 days ago'), ('헤라 너무 좋아하는 브랜드라서 이거 거의 나오자마자 샀는데 저하곤 안맞아서 넘 슬퍼요(수부지, 홍조, 모공 있는 피부) 무너질때 모공에 심하게 끼고 시간지나니까 너무 건조해요 손등말고 피부에 테스트 가능하면 꼭 해보시고 사세요. 손등엔 얼굴과는 달리 진짜 이쁘게 표현되요 ', '6 days ago (edited)'), ('이번 헤라쿠션 진짜 탐나요 ㅎㅎㅎㅎ 베이스 제품 너무많아서 그만 사야하는데..ㅜ 하이라이터 용인 0호가 너무 끌리네요 ㅋㅋㅋㅋㅋㅋㅋ', '6 days ago'), ('저는 건성 피부이고 언니처럼 광이 필요한 사람으로서 에스쁘아 비글로우와 함께 헤라 스킨 레디언스 쿠션을 넘나 사랑하는데 이번에 새롭게 업그레이드 됐다는 말 듣고 언니의 리뷰를 기다렸는데 나이스 타이밍~! 당장 데리고 가야겠네요~!', '6 days ago'), ('

In [20]:
transcript_music_data[0]

['이 제품만큼은 백화점에서 사도 후회\n안 한다 맑은 피부편 좋아한다 하시는\n분들은 꼭 사용해 보셨으면 좋겠는\n제품 제 요즘 1등 블러셔입니다이\n컬러가 진짜 기가 많아 진짜 제가\n사용해 본 펜슬 중에 지속력\n1등들입니다 하\n나에요 자봐 오늘은 오랜만에 백화점\n추천템 영상을 준비를 해 봤는데요\n오늘 소개해드릴 제품들은 시중에 진짜\n좋은 제품들 너무너무 많잖아요\n그중에서이 제품만큼은 100 화점에서\n사도 후회 안 한다 돈값 제대로 한다\n뽕 제대로 뽑을 수 있다 하는\n제품들만 모아모아 가지고 왔습니다\n제가 진짜 오랜 시간 동안 꾸준히\n사용한 제품들만 가지고 왔거든요\n그래서 오늘 소개해 드리는 제품들\n끝까지 집중해서 함께해 주셨으면\n좋겠어요 그럼 오늘 영상도 같이\n시작하러\n가보실까요 먼저 첫 번째 제품은\n시세이도 파운데이션 제품인데요이 제품\n한동안 핫했던 제품이죠 제가 사용해\n본 파운데이션 중에이 정도로 초\n촉촉하고 시스루한 파운데이션은 없었다\n진짜 유명가 파는 고런 제품이에요\n모공 주름 사이사이를 얇게 싹 메꿔\n주면서 글로원 물광이 밀착시킬수록 쫙\n올라와요 그리고 제형이 차곡차곡 진짜\n잘 쌓이는 제형이라 두 세겹 쌓아\n줘도 전혀 < datun 보이지 않고\n맑게 투명하지만 매끈한 피부로 만들어\n주는 진짜 돈값 제대로 하는\n파운데이션이 그든 그래서 특히\n건성분들 투명한 피부편 좋아한다\n하시는 분들에게 정말 강추드리는\n제품이에요 지속력도 다크니 거의 없는\n편이고 얘가 워낙 얇고 투명하고\n시스루한 파운데이션이라고 했잖아요\n그래서 시간이 지났을 때 내 피부의\n유분이랑 자연스럽게 섞이면서 오히려\n약간 더 윤광이 올라오고 좀 더\n투명한 느낌으로 내추럴하게 자연스럽게\n좀 먹으면 그런 느낌으로 예쁘게\n무너져 가지고 약간 파운데이션이 살짝\n사라졌다 그런 느낌만나요 무너졌다\n지저분하다 그런 느낌이 아니라 좀\n시간 지나서 파운데이션이 살짝\n날아가고 내 피부의 유분감이\n자연스럽게 어우러지고 있다 윤곽